In [1]:
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.ml.recommendation import ALS



from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.session import SparkSession


from pyspark import SparkContext

sc = SparkContext()
sqlContext = SQLContext(sc)


spark = SparkSession(sc)






In [34]:
rdd = sc.textFile('datasets/Users_Rating_Data.csv')
rawdata = rdd.map(lambda x: (int(x.split(",")[0]) , int(x.split(",")[1]),int(x.split(",")[2]))).filter(lambda x: x[0] != 'airline_id')


rawdataframe = sqlContext.createDataFrame(rawdata, ['airline_id',"user_id","rating"])
rawdataframe = rawdataframe.orderBy(rand())
rawdataframe.show()


+----------+-------+------+
|airline_id|user_id|rating|
+----------+-------+------+
|         1|  13366|     3|
|         1|  12693|     3|
|         1|  10785|     2|
|         1|  12387|     2|
|         3|  12128|     8|
|         1|   6206|     2|
|         3|  11946|     9|
|         1|   6616|     3|
|         1|   1389|     8|
|         2|  12391|     2|
|         1|   8624|     1|
|         2|   5242|    10|
|         2|   3683|     2|
|         3|   5806|     2|
|         3|  11392|    10|
|         2|  12289|     1|
|         2|   8210|     8|
|         2|   1093|     0|
|         1|   8428|     3|
|         1|  12362|     1|
+----------+-------+------+
only showing top 20 rows



In [35]:
rawdataframe.head()
rawdataframe.describe().show()

+-------+------------------+-----------------+------------------+
|summary|        airline_id|          user_id|            rating|
+-------+------------------+-----------------+------------------+
|  count|            179976|           179976|            179976|
|   mean|               2.0|7499.499933324443| 3.497605236253723|
| stddev|0.8164988492856836|4330.331511089922|3.1292329290142926|
|    min|                 1|                0|                 0|
|    max|                 3|            14999|                10|
+-------+------------------+-----------------+------------------+



In [36]:
(training, test) = rawdataframe.randomSplit([0.8, 0.2])


In [42]:
als = ALS(maxIter=10, regParam=0.02, userCol="user_id", itemCol="airline_id", ratingCol="rating",coldStartStrategy="drop",nonnegative=True)
model = als.fit(training)

In [43]:
predictions = model.transform(test)
predictions.show()

+----------+-------+------+----------+
|airline_id|user_id|rating|prediction|
+----------+-------+------+----------+
|         1|    148|     5| 6.7196846|
|         1|    471|    10| 6.9970956|
|         1|    496|     7|  5.711477|
|         1|   1088|     5|  6.341976|
|         1|   1645|     8|  7.646962|
|         1|   2122|     5| 7.9662423|
|         1|   2366|    10| 7.2921357|
|         1|   3749|     8| 6.7230973|
|         1|   3794|     8|  7.648612|
|         1|   3918|     7|  6.086511|
|         1|   3918|    10|  6.086511|
|         1|   3997|     5|  7.292645|
|         1|   4101|     7|  7.967496|
|         1|   4519|     5| 6.3930793|
|         1|   4818|    10|  7.013397|
|         1|   4900|     7|  8.245024|
|         1|   4935|     9|   7.65016|
|         1|   5156|     0| 0.6502005|
|         1|   5518|     0| 1.4087081|
|         1|   5518|     2| 1.4087081|
+----------+-------+------+----------+
only showing top 20 rows



In [44]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.6555524869946365


In [45]:
single_user = test.filter(test['user_id']==471).select(['airline_id','user_id','rating'])
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+----------+-------+------+
|airline_id|user_id|rating|
+----------+-------+------+
|         1|    471|    10|
|         3|    471|     2|
|         3|    471|     3|
+----------+-------+------+



In [46]:
recomendations = model.transform(single_user)
recomendations.orderBy('prediction',ascending=False).show()

+----------+-------+------+----------+
|airline_id|user_id|rating|prediction|
+----------+-------+------+----------+
|         1|    471|    10| 6.9970956|
|         3|    471|     2|0.34230512|
|         3|    471|     3|0.34230512|
+----------+-------+------+----------+



In [ ]:




+----------+-------+------+
|airline_id|user_id|rating|
+----------+-------+------+
|         1|  13366|     3|
|         1|  12693|     3|
|         1|  10785|     2|
|         1|  12387|     2|
|         3|  12128|     8|
|         1|   6206|     2|
|         3|  11946|     9|
|         1|   6616|     3|
|         1|   1389|     8|
|         2|  12391|     2|
|         1|   8624|     1|
|         2|   5242|    10|
|         2|   3683|     2|
|         3|   5806|     2|
|         3|  11392|    10|
|         2|  12289|     1|
|         2|   8210|     8|
|         2|   1093|     0|
|         1|   8428|     3|
|         1|  12362|     1|
+----------+-------+------+